In [ ]:
import faiss
d = 768 # dimensions of nomic embedding, the embedding model that we're going to use
faiss_index = faiss.IndexFlatL2(d)

In [ ]:
import os
import openai
os.environ['OPENAI_API_KEY'] = 'lm-studio'
os.environ['OPENAI_API_BASE'] = 'http://localhost:1234/v1'

openai.api_key = os.getenv('OPENAI_API_KEY')
openai.api_base = os.getenv('OPENAI_API_BASE')

In [ ]:
from llama_index import Document
import pandas as pd
from llama_index import VectorStoreIndex, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore

# Load CSV data
with open(r'GPT\data\gpt_data.csv', 'r', encoding='utf-8', errors='replace') as file:
    df = pd.read_csv(file)

df['LLM Output']=None
for i, row in df.iterrows(): 
    faiss_index = faiss.IndexFlatL2(d)
    text = str(row['Incident Narrative'])
    text_list=[text]
    print(text_list)
    documents = [Document(text=t) for t in text_list]
    
    vector_store = FaissVectorStore(faiss_index=faiss_index)
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context, show_progress=True)
    query = """Your task is to classify each news article into one of the following categories. The output should only contain the category name, with no explanation and other output: 
    Unlawful detention - Refers to refers to the act of detaining or confining an individual without legal justification or due process. For example, if protesters are arrested and detained without legal basis during peaceful demonstrations, with no access to legal representation, this would be considered unlawful detention. 

 

    Human trafficking - Refers to the act of recruiting, transporting, transferring, harboring, or receiving individuals through force, fraud, coercion, or other forms of deception for the purpose of exploitation. Exploitation can take many forms, including forced labor, sexual exploitation, slavery, servitude, or the removal of organs. It is considered a severe violation of human rights and is illegal under international and domestic laws. If an incident is considered Human trafficking it would also be considered as a War crime. 

    

    Enslavement - refers to the act of exercising ownership or control over another person, treating them as property, and depriving them of their freedom. It often involves forcing individuals to perform labor or services under coercion, violence, or the threat of punishment. If an incident is considered Enslavement, it would also be considered as a War crime. 

    

    Willful killing of civilians - Refers to the intentional killing of civilians who are not directly participating in hostilities, with full knowledge of their noncombatant status. This includes acts like massacres, executions, or deliberate bombings of civilian sites such as homes, schools, or hospitals, where the clear intent is to cause death. For example, a military unit massacring the residents of a village. 

    

    Mass execution - Refers to the deliberate killing of a large scale number of individuals, often carried out by state or non-state actors as part of systematic persecution, acts of war, or punitive measures. The victims are typically selected based on political, ethnic, religious, or social affiliations, and the killings are often premeditated and organized. If an incident is considered Mass execution, it would also be considered as a War crime. 

    

    Kidnapping - Refers to the unlawful and intentional abduction, confinement, or holding of an individual against their will, often for a specific purpose such as extortion, ransom, political leverage, forced labor, or exploitation. It is a serious crime and violates the individual's right to freedom and security. 
    

    Extrajudicial killing - Refers to the killing of a person without any legal process, such as arrest, trial, or sentencing. It is carried out outside the law, often by state agents or with their approval. 

    

    Forced disappearance - Refers the act of abducting or detaining a person against their will, followed by a refusal to disclose their fate or whereabouts. This leaves the victim outside the protection of the law and often causes anguish to their family and community. 

    

    Damage or destruction of civilian critical infrastructure - Refers to the reckless harm, sabotage, or destruction of essential facilities, systems, or services necessary for the well-being, safety, and survival of civilian populations. This includes infrastructure such as hospitals, water supplies, power grids, schools, transportation systems, and communication networks. 

    

    Damage or destruction, looting, or theft of cultural heritage - Refers to the harm, removal, or appropriation of culturally significant sites, objects, or artifacts during conflicts, disasters, or other destabilizing events. These acts violate international laws that protect cultural heritage as part of humanity's shared history and identity. Furthermore, this also refers to looting incidents. 

    

    Military operations (battle, shelling) - Refers to actions explicitly conducted between opposing armed forces, such as the RSF and SAF, during a conflict or war. These actions involve the use of weapons, strategies, and tactics to achieve military objectives, focusing on direct engagements or operations targeting enemy positions. Narratives mentioning attacks on civilian areas or indiscriminate shelling are not included in this category, even if long-range weapons or artillery are used. 
    

    Gender-based or other conflict-related sexual violence - Refers to acts of sexual violence committed during or as a result of armed conflict, often targeting individuals based on their gender, identity, or perceived vulnerability. Incidents such as rape or sexual harassment are considered Gender-based or other conflict-related sexual violence. 



    Violent crackdowns on protesters/opponents/civil rights abuse - Refers to the use of excessive or unlawful force suppress dissent, silence opposition. These acts often involve targeting individuals or groups engaging in protests, political opposition, or advocacy for civil rights. 

    

    Indiscriminate use of weapons - Refers to the use of weapons, such as shelling or bombing, in a manner that impacts buildings, neighborhoods, or areas without clear differentiation between combatants and civilians, or military and civilian infrastructure. This category applies only to incidents involving the use of explosives or long-range weapons that cause widespread harm or destruction, regardless of whether brute force or a massacre is involved, unless explicitly mentioned. 

    

    Torture or indications of torture - Refers to the infliction of severe physical or psychological pain and suffering on a person, typically to punish, intimidate, extract information, or coerce. 

    

    Persecution based on political, racial, ethnic, gender, or sexual orientation - Refers to the systematic mistreatment, harassment, or oppression of individuals or groups due to their political beliefs, race, ethnicity, gender identity, or sexual orientation. 

    

    Movement of military, paramilitary, or other troops and equipment - Refers to the deployment, transfer, or relocation of armed forces, armed groups, or their equipment as part of strategic or operational objectives. This movement may occur during preparation for conflict, active military operations, or in maintaining a presence in certain areas. 

 
    Classify each news article into one of the above categories. Note that a single article can be classified into only one category based on the most prominent theme. If an article fits into multiple categories, select the one that best describes the primary issue."""
    response = index.as_query_engine(similarity_top_k=1).query(query)

    article_metadata_detected_str = response.response

    print(str(i) + " "+ article_metadata_detected_str)
    df.at[i,'LLM Output'] = article_metadata_detected_str
df.to_csv('Sudan_Conflict_SLC_Def.csv', index=False, encoding='utf-8')